# Chapter 28: Type-Safe Event Handling

---

## 28.1 Event Emitter Pattern

The Event Emitter pattern is a foundational design pattern for building loosely coupled, event-driven architectures. TypeScript's type system elevates this pattern by ensuring that event names and their associated payloads are type-safe, preventing common runtime errors like misspelled event names or incorrect payload structures.

### 28.1.1 Basic Typed Event Emitter

A type-safe event emitter uses generics to enforce that event names exist in a predefined map and that their payloads match the expected types.

```typescript
// Basic typed event emitter
class TypedEventEmitter<EventMap extends Record<string, any>> {
  private listeners: {
    [K in keyof EventMap]?: Array<(payload: EventMap[K]) => void>;
  } = {};

  // Subscribe to an event
  on<K extends keyof EventMap>(
    event: K,
    listener: (payload: EventMap[K]) => void
  ): () => void {
    if (!this.listeners[event]) {
      this.listeners[event] = [];
    }
    this.listeners[event]!.push(listener);

    // Return unsubscribe function
    return () => this.off(event, listener);
  }

  // Unsubscribe from an event
  off<K extends keyof EventMap>(
    event: K,
    listener: (payload: EventMap[K]) => void
  ): void {
    const eventListeners = this.listeners[event];
    if (eventListeners) {
      const index = eventListeners.indexOf(listener);
      if (index > -1) {
        eventListeners.splice(index, 1);
      }
    }
  }

  // Emit an event with typed payload
  emit<K extends keyof EventMap>(event: K, payload: EventMap[K]): void {
    const eventListeners = this.listeners[event];
    if (eventListeners) {
      eventListeners.forEach(listener => listener(payload));
    }
  }

  // Subscribe once
  once<K extends keyof EventMap>(
    event: K,
    listener: (payload: EventMap[K]) => void
  ): void {
    const onceListener = (payload: EventMap[K]) => {
      this.off(event, onceListener);
      listener(payload);
    };
    this.on(event, onceListener);
  }
}

// Usage
interface UserEvents {
  userCreated: { id: number; name: string; email: string };
  userUpdated: { id: number; changes: Partial<{ name: string; email: string }> };
  userDeleted: { id: number };
  userLogin: { userId: number; timestamp: Date };
}

const emitter = new TypedEventEmitter<UserEvents>();

// TypeScript provides autocomplete and type checking
emitter.on("userCreated", (data) => {
  console.log(data.name); // ✅ TypeScript knows data has name property
  console.log(data.id);   // ✅ And id property
});

// emitter.on("userCreated", (data) => {
//   console.log(data.phone); // ❌ Error: phone doesn't exist on payload
// });

// emitter.emit("userCreated", { id: 1 }); // ❌ Error: missing name and email

emitter.emit("userCreated", {
  id: 1,
  name: "John",
  email: "john@example.com"
}); // ✅ Valid
```

### 28.1.2 Advanced Event Emitter Features

Extend the basic pattern with advanced features like event namespaces, wildcards, and async listeners.

```typescript
class AdvancedEventEmitter<EventMap extends Record<string, any>> {
  private listeners: Map<keyof EventMap, Set<Function>> = new Map();
  private wildcards: Set<(event: string, payload: any) => void> = new Set();

  // Standard event subscription
  on<K extends keyof EventMap>(
    event: K,
    listener: (payload: EventMap[K]) => void | Promise<void>
  ): () => void {
    if (!this.listeners.has(event)) {
      this.listeners.set(event, new Set());
    }
    this.listeners.get(event)!.add(listener);

    return () => this.off(event, listener);
  }

  // Listen to all events (wildcard)
  onAny(listener: <K extends keyof EventMap>(event: K, payload: EventMap[K]) => void): () => void {
    this.wildcards.add(listener as Function);
    return () => this.wildcards.delete(listener as Function);
  }

  // Emit with async support
  async emit<K extends keyof EventMap>(event: K, payload: EventMap[K]): Promise<void> {
    const promises: Promise<void>[] = [];

    // Call specific listeners
    const eventListeners = this.listeners.get(event);
    if (eventListeners) {
      eventListeners.forEach(listener => {
        const result = listener(payload);
        if (result instanceof Promise) {
          promises.push(result);
        }
      });
    }

    // Call wildcards
    this.wildcards.forEach(listener => {
      const result = listener(event, payload);
      if (result instanceof Promise) {
        promises.push(result);
      }
    });

    await Promise.all(promises);
  }

  // Emit with error handling
  async emitSafe<K extends keyof EventMap>(
    event: K,
    payload: EventMap[K]
  ): Promise<{ success: number; failed: number; errors: Error[] }> {
    const results = { success: 0, failed: 0, errors: [] as Error[] };

    const eventListeners = this.listeners.get(event);
    if (eventListeners) {
      for (const listener of eventListeners) {
        try {
          await listener(payload);
          results.success++;
        } catch (error) {
          results.failed++;
          results.errors.push(error as Error);
        }
      }
    }

    return results;
  }

  // Remove all listeners for an event
  removeAllListeners<K extends keyof EventMap>(event?: K): void {
    if (event) {
      this.listeners.delete(event);
    } else {
      this.listeners.clear();
      this.wildcards.clear();
    }
  }

  // Get listener count
  listenerCount<K extends keyof EventMap>(event: K): number {
    return this.listeners.get(event)?.size ?? 0;
  }
}

// Usage with async handlers
interface AppEvents {
  dataReceived: { data: string; timestamp: number };
  error: { message: string; code: number };
  shutdown: { reason: string };
}

const appEmitter = new AdvancedEventEmitter<AppEvents>();

// Async handler
appEmitter.on("dataReceived", async (payload) => {
  await processData(payload.data);
});

// Wildcard listener
appEmitter.onAny((event, payload) => {
  console.log(`Event ${String(event)} fired with:`, payload);
});

// Safe emit with error tracking
appEmitter.emitSafe("dataReceived", {
  data: "test",
  timestamp: Date.now()
}).then(results => {
  if (results.failed > 0) {
    console.error("Some handlers failed:", results.errors);
  }
});
```

---

## 28.2 Typed Event Maps

Event maps are the foundation of type-safe event handling. They define the contract between event emitters and listeners, ensuring that every event name has a corresponding, well-defined payload structure.

### 28.2.1 Defining Event Maps

Create comprehensive event maps that describe all possible events in your application domain.

```typescript
// Domain-specific event maps
interface UserDomainEvents {
  "user:created": {
    userId: string;
    email: string;
    timestamp: Date;
  };
  "user:updated": {
    userId: string;
    changes: Array<{ field: string; oldValue: any; newValue: any }>;
    updatedBy: string;
  };
  "user:deleted": {
    userId: string;
    reason: string;
    deletedAt: Date;
  };
  "user:login": {
    userId: string;
    ip: string;
    userAgent: string;
  };
  "user:logout": {
    userId: string;
    sessionDuration: number;
  };
}

interface OrderDomainEvents {
  "order:created": {
    orderId: string;
    userId: string;
    items: Array<{ productId: string; quantity: number; price: number }>;
    total: number;
  };
  "order:status:changed": {
    orderId: string;
    oldStatus: "pending" | "processing" | "shipped" | "delivered";
    newStatus: "pending" | "processing" | "shipped" | "delivered";
    changedAt: Date;
  };
  "order:cancelled": {
    orderId: string;
    reason: string;
    refunded: boolean;
  };
}

// Combine multiple event maps
type AppEvents = UserDomainEvents & OrderDomainEvents;

// Type helper to extract event names
type EventNames<T> = keyof T & string;

// Type helper to extract payload for specific event
type EventPayload<T, K extends keyof T> = T[K];

// Usage
type AllEvents = EventNames<AppEvents>; // "user:created" | "user:updated" | ...
type CreatedPayload = EventPayload<AppEvents, "user:created">;
```

### 28.2.2 Hierarchical Event Types

Organize events hierarchically for better organization and type inference.

```typescript
// Base event interface
interface BaseEvent {
  timestamp: Date;
  correlationId: string;
}

// Specific event types extending base
interface UserCreatedEvent extends BaseEvent {
  type: "user.created";
  payload: {
    userId: string;
    email: string;
  };
}

interface UserUpdatedEvent extends BaseEvent {
  type: "user.updated";
  payload: {
    userId: string;
    changes: Record<string, { old: any; new: any }>;
  };
}

// Discriminated union of events
type UserEvent = UserCreatedEvent | UserUpdatedEvent;

// Type-safe handler
function handleUserEvent(event: UserEvent) {
  switch (event.type) {
    case "user.created":
      // TypeScript knows event.payload has userId and email
      console.log(event.payload.email);
      break;
    case "user.updated":
      // TypeScript knows event.payload has changes
      console.log(event.payload.changes);
      break;
  }
}

// Event map from union
type EventMapFromUnion<T extends { type: string; payload: any }> = {
  [K in T as K["type"]]: K["payload"];
};

type UserEventMap = EventMapFromUnion<UserEvent>;
// { "user.created": { userId: string; email: string; }, "user.updated": { ... } }
```

### 28.2.3 Strict Event Validation

Implement runtime validation alongside compile-time type safety.

```typescript
// Runtime type validation with TypeScript
import { z } from "zod";

// Define schemas
const UserCreatedSchema = z.object({
  userId: z.string().uuid(),
  email: z.string().email(),
  timestamp: z.date()
});

const UserUpdatedSchema = z.object({
  userId: z.string().uuid(),
  changes: z.array(z.object({
    field: z.string(),
    oldValue: z.any(),
    newValue: z.any()
  })),
  updatedBy: z.string()
});

// Type inference from schemas
type UserCreatedPayload = z.infer<typeof UserCreatedSchema>;
type UserUpdatedPayload = z.infer<typeof UserUpdatedSchema>;

interface ValidatedEventMap {
  "user:created": UserCreatedPayload;
  "user:updated": UserUpdatedPayload;
}

class ValidatedEventEmitter<EventMap extends Record<string, any>> {
  private validators: Partial<{
    [K in keyof EventMap]: (payload: unknown) => payload is EventMap[K];
  }> = {};

  registerValidator<K extends keyof EventMap>(
    event: K,
    validator: (payload: unknown) => payload is EventMap[K]
  ): void {
    this.validators[event] = validator;
  }

  emit<K extends keyof EventMap>(event: K, payload: EventMap[K]): void {
    const validator = this.validators[event];
    if (validator && !validator(payload)) {
      throw new Error(`Invalid payload for event ${String(event)}`);
    }
    // ... emit logic
  }
}

// Setup
const validatedEmitter = new ValidatedEventEmitter<ValidatedEventMap>();
validatedEmitter.registerValidator("user:created", (payload): payload is UserCreatedPayload => {
  return UserCreatedSchema.safeParse(payload).success;
});
```

---

## 28.3 Type-Safe DOM Events

Handling browser DOM events with TypeScript requires understanding the built-in type definitions and creating wrappers that preserve type safety for custom events and event delegation.

### 28.3.1 Standard DOM Event Types

TypeScript provides comprehensive types for standard DOM events.

```typescript
// Event listener with proper types
const button = document.getElementById("myButton");

if (button) {
  // TypeScript knows the event type based on the event name
  button.addEventListener("click", (event) => {
    // event is automatically typed as MouseEvent
    console.log(event.clientX, event.clientY);
    console.log(event.target); // EventTarget | null
  });

  button.addEventListener("keydown", (event) => {
    // event is KeyboardEvent
    if (event.key === "Enter") {
      console.log("Enter pressed");
    }
  });

  // Input events
  const input = document.getElementById("myInput") as HTMLInputElement;
  input.addEventListener("input", (event) => {
    // event is InputEvent, but target gives us the element
    const value = (event.target as HTMLInputElement).value;
    console.log(value);
  });
}

// Generic event handler with type narrowing
function addTypedEventListener<K extends keyof HTMLElementEventMap>(
  element: HTMLElement,
  event: K,
  handler: (event: HTMLElementEventMap[K]) => void
): void {
  element.addEventListener(event, handler as EventListener);
}

// Usage with full type safety
const div = document.createElement("div");
addTypedEventListener(div, "mouseenter", (e) => {
  // e is MouseEvent
  console.log(e.relatedTarget);
});
```

### 28.3.2 Custom DOM Events

Create and dispatch custom events with typed payloads.

```typescript
// Define custom event detail types
interface UserLoginDetail {
  userId: string;
  username: string;
  timestamp: number;
}

interface DataLoadedDetail<T> {
  data: T;
  source: string;
}

// Extend HTMLElementEventMap for global awareness
declare global {
  interface HTMLElementEventMap {
    "user:login": CustomEvent<UserLoginDetail>;
    "data:loaded": CustomEvent<DataLoadedDetail<unknown>>;
  }
}

// Type-safe custom event dispatcher
function dispatchCustomEvent<T>(
  target: HTMLElement,
  eventName: string,
  detail: T,
  options?: CustomEventInit
): void {
  const event = new CustomEvent(eventName, {
    detail,
    bubbles: true,
    cancelable: true,
    ...options
  });
  target.dispatchEvent(event);
}

// Usage
const app = document.getElementById("app")!;

// Dispatch
dispatchCustomEvent<UserLoginDetail>(app, "user:login", {
  userId: "123",
  username: "john",
  timestamp: Date.now()
});

// Listen
app.addEventListener("user:login", (event) => {
  // event.detail is typed as UserLoginDetail
  console.log(event.detail.username);
});
```

### 28.3.3 Event Delegation

Implement type-safe event delegation for dynamic content.

```typescript
class DelegatedEventHandler<EventMap extends Record<string, any>> {
  private container: HTMLElement;
  private listeners: Map<string, Set<(event: Event, target: HTMLElement) => void>> = new Map();

  constructor(container: HTMLElement) {
    this.container = container;
    this.setupDelegation();
  }

  private setupDelegation(): void {
    this.container.addEventListener("click", (e) => this.handleEvent("click", e));
    this.container.addEventListener("submit", (e) => this.handleEvent("submit", e));
    // Add other event types as needed
  }

  private handleEvent(type: string, event: Event): void {
    const handlers = this.listeners.get(type);
    if (!handlers) return;

    const target = event.target as HTMLElement;
    
    handlers.forEach(handler => {
      handler(event, target);
    });
  }

  on<K extends keyof EventMap>(
    event: K,
    selector: string,
    handler: (payload: EventMap[K], target: HTMLElement) => void
  ): void {
    const eventName = String(event);
    if (!this.listeners.has(eventName)) {
      this.listeners.set(eventName, new Set());
    }

    const wrappedHandler = (event: Event, target: HTMLElement) => {
      const matchingTarget = target.closest(selector);
      if (matchingTarget) {
        handler(event as EventMap[K], matchingTarget as HTMLElement);
      }
    };

    this.listeners.get(eventName)!.add(wrappedHandler);
  }

  destroy(): void {
    this.listeners.clear();
  }
}

// Usage
interface TableEvents {
  "row:click": MouseEvent;
  "cell:edit": CustomEvent<{ column: string; value: string }>;
}

const table = document.getElementById("dataTable")!;
const handler = new DelegatedEventHandler<TableEvents>(table);

handler.on("row:click", "tr", (event, target) => {
  // event is MouseEvent, target is the TR element
  console.log("Row clicked:", target.dataset.id);
});

handler.on("cell:edit", "td", (event, target) => {
  console.log("Cell edit:", event.detail);
});
```

---

## 28.4 Custom Event Systems

Building domain-specific event architectures that go beyond simple pub/sub, including event sourcing, CQRS patterns, and reactive streams.

### 28.4.1 Event Sourcing

Implement event sourcing patterns where state is derived from a sequence of events.

```typescript
// Event store with type safety
interface EventStore<EventMap extends Record<string, any>> {
  append<K extends keyof EventMap>(
    aggregateId: string,
    event: K,
    payload: EventMap[K],
    metadata?: Record<string, any>
  ): Promise<void>;

  getEvents(aggregateId: string): Promise<Array<{
    event: keyof EventMap;
    payload: EventMap[keyof EventMap];
    timestamp: Date;
    version: number;
  }>>;

  subscribe<K extends keyof EventMap>(
    event: K,
    handler: (payload: EventMap[K], metadata: { aggregateId: string; version: number }) => void
  ): () => void;
}

// Aggregate root with event sourcing
abstract class AggregateRoot<EventMap extends Record<string, any>, State> {
  protected state: State;
  protected version: number = 0;
  private uncommittedEvents: Array<{ event: keyof EventMap; payload: any }> = [];

  constructor(initialState: State) {
    this.state = initialState;
  }

  protected emit<K extends keyof EventMap>(event: K, payload: EventMap[K]): void {
    this.uncommittedEvents.push({ event, payload });
    this.applyEvent(event, payload);
    this.version++;
  }

  protected abstract applyEvent<K extends keyof EventMap>(
    event: K,
    payload: EventMap[K]
  ): void;

  getUncommittedEvents(): Array<{ event: keyof EventMap; payload: any }> {
    return [...this.uncommittedEvents];
  }

  markCommitted(): void {
    this.uncommittedEvents = [];
  }

  loadFromHistory(events: Array<{ event: keyof EventMap; payload: any }>): void {
    events.forEach(({ event, payload }) => {
      this.applyEvent(event, payload);
      this.version++;
    });
  }
}

// Concrete implementation
interface OrderEvents {
  "order:created": { items: string[]; total: number };
  "order:item:added": { productId: string; quantity: number; price: number };
  "order:item:removed": { productId: string };
  "order:shipped": { trackingNumber: string };
}

interface OrderState {
  items: Array<{ productId: string; quantity: number; price: number }>;
  total: number;
  status: "pending" | "shipped" | "delivered";
  trackingNumber?: string;
}

class Order extends AggregateRoot<OrderEvents, OrderState> {
  constructor(id: string) {
    super({
      items: [],
      total: 0,
      status: "pending"
    });
  }

  addItem(productId: string, quantity: number, price: number): void {
    this.emit("order:item:added", { productId, quantity, price });
  }

  ship(trackingNumber: string): void {
    if (this.state.status !== "pending") {
      throw new Error("Order already shipped");
    }
    this.emit("order:shipped", { trackingNumber });
  }

  protected applyEvent<K extends keyof OrderEvents>(
    event: K,
    payload: OrderEvents[K]
  ): void {
    switch (event) {
      case "order:item:added":
        const { productId, quantity, price } = payload;
        this.state.items.push({ productId, quantity, price });
        this.state.total += quantity * price;
        break;
      case "order:shipped":
        this.state.status = "shipped";
        this.state.trackingNumber = payload.trackingNumber;
        break;
    }
  }
}
```

### 28.4.2 Reactive Event Streams

Integrate with reactive programming patterns for event streams.

```typescript
import { Observable, Subject, filter, map } from "rxjs";

// Reactive event bus
class ReactiveEventBus<EventMap extends Record<string, any>> {
  private stream = new Subject<{
    event: keyof EventMap;
    payload: EventMap[keyof EventMap];
  }>();

  // Emit events into the stream
  emit<K extends keyof EventMap>(event: K, payload: EventMap[K]): void {
    this.stream.next({ event, payload });
  }

  // Subscribe to specific event as Observable
  on<K extends keyof EventMap>(event: K): Observable<EventMap[K]> {
    return this.stream.pipe(
      filter(e => e.event === event),
      map(e => e.payload as EventMap[K])
    );
  }

  // Subscribe to multiple events
  onMany<K extends keyof EventMap>(events: K[]): Observable<{ event: K; payload: EventMap[K] }> {
    return this.stream.pipe(
      filter(e => events.includes(e.event as K)),
      map(e => ({ event: e.event as K, payload: e.payload as EventMap[K] }))
    );
  }

  // Pipe transformations
  pipe<T>(...operations: any[]): Observable<T> {
    return this.stream.pipe(...operations);
  }
}

// Usage with RxJS operators
interface StreamEvents {
  "data:received": { id: string; content: string };
  "data:processed": { id: string; result: any };
  "error:occurred": { error: Error; context: string };
}

const eventBus = new ReactiveEventBus<StreamEvents>();

// Subscribe with RxJS operators
eventBus
  .on("data:received")
  .pipe(
    filter(data => data.content.length > 0),
    map(data => ({ ...data, timestamp: Date.now() }))
  )
  .subscribe(processed => {
    console.log("Received:", processed);
  });

// Error handling
eventBus.on("error:occurred").subscribe(({ error, context }) => {
  console.error(`Error in ${context}:`, error);
});
```

---

## 28.5 Chapter Summary and Exercises

### Chapter Summary

In this chapter, we explored type-safe event handling patterns in TypeScript:

**Key Takeaways:**

1. **Typed Event Emitters**:
   - Use generics to map event names to payload types
   - Implement subscribe, unsubscribe, and emit methods
   - Support async handlers and error boundaries

2. **Event Maps**:
   - Define comprehensive event contracts
   - Use discriminated unions for event variants
   - Implement runtime validation alongside types

3. **DOM Event Handling**:
   - Leverage built-in TypeScript DOM types
   - Create custom events with typed payloads
   - Implement type-safe event delegation

4. **Custom Architectures**:
   - Event sourcing patterns with aggregate roots
   - Reactive streams with RxJS integration
   - Hierarchical event organization

### Practical Exercises

**Exercise 1: Basic Event Emitter**

Implement a type-safe event emitter:

```typescript
// Create a TypedEventEmitter class that:
// 1. Uses a generic EventMap to type events
// 2. Implements on(), off(), and emit() methods
// 3. Supports once() for one-time listeners
// 4. Returns unsubscribe function from on()
// 5. Handles errors in listeners without crashing

interface MyEvents {
  "user:login": { userId: string; timestamp: number };
  "user:logout": { userId: string; duration: number };
  "error": { message: string; stack?: string };
}

const emitter = new TypedEventEmitter<MyEvents>();
// Test with all methods
```

**Exercise 2: Event Sourcing**

Build an event sourcing system:

```typescript
// Create an EventSourcedAggregate base class that:
// 1. Stores uncommitted events
// 2. Has applyEvent() method for state changes
// 3. Supports loading from history
// 4. Tracks version numbers

// Implement a concrete ShoppingCart aggregate with events:
// - cart:created
// - cart:item:added
// - cart:item:removed
// - cart:checked:out

// Ensure type safety throughout the event sourcing logic
```

**Exercise 3: DOM Event Wrapper**

Create type-safe DOM event utilities:

```typescript
// Create utilities that:
// 1. addTypedListener() - wrapper around addEventListener with proper typing
// 2. createCustomEvent() - factory for custom events with typed detail
// 3. delegate() - event delegation with selector matching and type safety

// Test with various DOM events and custom events
// Ensure autocomplete works for event names
```

**Exercise 4: Reactive Integration**

Integrate with RxJS:

```typescript
// Extend the event emitter to work with RxJS:
// 1. Convert event subscriptions to Observables
// 2. Use RxJS operators (filter, map, debounce)
// 3. Create event streams that combine multiple event types
// 4. Implement error handling and retry logic

// Create a search component that:
// - Emits "search:input" events
// - Debounces them
// - Emits "search:execute" after debounce
// - Handles "search:results" and "search:error"
```

### Additional Resources

- **TypeScript Event Handling**: https://www.typescriptlang.org/docs/handbook/dom-manipulation.html
- **Event Emitter Pattern**: https://refactoring.guru/design-patterns/observer
- **RxJS with TypeScript**: https://rxjs.dev/guide/overview
- **Event Sourcing**: https://martinfowler.com/eaaDev/EventSourcing.html

---

## Coming Up Next: Chapter 29 - Type-Safe API Clients

In the next chapter, we will explore **Type-Safe API Clients**, building robust HTTP clients with TypeScript:

- **29.1 Typing HTTP Responses** - Handling API response types
- **29.2 Type-Safe Request Builders** - Building requests with type safety
- **29.3 Handling API Errors** - Error boundary types and handling
- **29.4 API Client Patterns** - Repository pattern, service layers, and more

Type-safe API clients ensure that your frontend and backend contracts remain synchronized, catching API mismatches at compile time rather than runtime.